<a href="https://colab.research.google.com/github/jbrenorv/15-puzzle/blob/main/15puzzle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from heapq import heappush, heappop
from bisect import bisect, insort
from copy import deepcopy
import base64

In [ ]:
META = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,0]
POS = [(0,0),(0,1),(0,2),(0,3),(1,0),(1,1),(1,2),(1,3),(2,0),(2,1),(2,2),(2,3),(3,0),(3,1),(3,2),(3,3)]
movements_by_position = ['rb','rbl','rbl','bl','rbt','rblt','rblt','blt','rbt','rblt','rblt','blt','rt','rlt','rlt','lt']
delta = {'r': 1, 'b': 4, 'l': -1, 't': -4}

In [ ]:
def get_num_of_inversions(arr):
  """
  Inversion count in Array using Merge Sort
  Reference: https://www.geeksforgeeks.org/inversion-count-in-array-using-merge-sort/
  """

  A = deepcopy(arr)
  A.remove(0)

  N = len(A)
  if N <= 1:
      return 0

  sortList = []
  result = 0

  # Heapsort, O(N*log(N))
  for i, v in enumerate(A):
      heappush(sortList, (v, i))

  # Create a sorted list of indexes
  x = []
  while sortList:

      # O(log(N))
      v, i = heappop(sortList)

      # Find the current minimum's index
      # the index y can represent how many minimums on the left
      y = bisect(x, i)

      # i can represent how many elements on the left
      # i - y can find how many bigger nums on the left
      result += i - y

      insort(x, i)

  return result

In [ ]:
def manhattan(board):
  dist = 0
  for i,e in enumerate(board):
    if e == 0:
      continue

    x1, y1 = POS[i]
    x2, y2 = POS[e - 1]
    dist += abs(x1 - x2) + abs(y1 - y2)

  return dist

In [ ]:
def linear_conflict(board):
  sz = 4
  conflicts = 0
  row = lambda i: POS[i][0]
  col = lambda i: POS[i][1]
  is_correct_row = lambda i,e: row(i) == row(e - 1)
  is_correct_col = lambda i,e: col(i) == col(e - 1)

  for a in range(sz):
    for b in range(sz):

      r, c = a, b
      i = r * sz + c
      if is_correct_row(i, board[i]) and board[i] != 0:
        for c1 in range(c, sz):
          j = r * sz + c1
          if is_correct_row(j, board[j]) and board[i] > board[j] and board[j] != 0:
            conflicts += 1

      r, c = b, a
      i = r * sz + c
      if is_correct_col(i, board[i]) and board[i] != 0:
        for r1 in range(r, sz):
          j = r1 * sz + c
          if is_correct_col(j, board[j]) and board[i] > board[j] and board[j] != 0:
            conflicts += 1

  return manhattan(board) + conflicts * 2;

In [ ]:
heuristcs = {
  'manhattan': manhattan,
  'linear_conflict': linear_conflict,
}

In [ ]:
class PriorityQueue:

  def __init__(self):
    self.array = []


  def empty(self):
    return len(self.array) == 0


  def find_priority(self, priority):
    start = 0
    end = len(self.array) -1
    while start <= end:
      mid = (start + end) // 2
      if self.array[mid][0] == priority:
        return (True, mid)
      if self.array[mid][0] > priority:
        start = mid + 1
      else:
        end = mid - 1
    return (False, start)


  def insert(self, element, priority):
    exists, index = self.find_priority(priority)
    if exists:
      self.array[index][1].append(element)
    else:
      self.array.insert(index, (priority, [element],))


  def min(self):
    if not self.empty():
      value = self.array[-1][1].pop()
      if len(self.array[-1][1]) == 0:
        self.array.pop()
      return value
    return None

In [ ]:
class PuzzleState:

  def __init__(self, board, parent=None, m=None):
    self.board = board;
    self.parent = parent;
    self.m = m
    self.g = parent.g + 1 if parent else 0;
    self.h = None


  def __eq__(self, other):
    return hasattr(other, 'board') and self.board == other.board


  def get_heuristc(self, heuristic):
    if not self.h:
      self.h = heuristcs[heuristic](self.board)
    return self.h


  def get_total_distance(self, heuristic):
    return self.g + self.get_heuristc(heuristic)


  def solvable(self):
    n_inv = get_num_of_inversions(self.board)
    i = self.board.index(0)
    e = (i // 4) +1
    return (n_inv + e) % 2 == 0


  def get_path(self):
    path = [None] * (self.g + 1)
    i = self.g
    state = self
    while state:
      path[i] = state
      state = state.parent
      i -= 1
    return path


  def get_hash(self):
    return base64.b64encode(bytes(self.board))


  def get_neighbors(self):

    def make_movement(i, d):
      j = i + delta[d]
      new = deepcopy(self.board)
      new[i], new[j] = new[j], new[i]
      return new

    neighbors = []
    i = self.board.index(0)

    for d in movements_by_position[i]:
      new = make_movement(i, d)
      neighbors.append(PuzzleState(new, self, d))

    return neighbors


  def solve(self, solver, heuristic='linear_conflict', log=True):
    if not self.solvable():
      if log:
        print('Impossible!')
      return (None, None,)

    answer, nb_tries = solver(self, heuristic)

    if log:
      print(f'Number of tries: {nb_tries}')
      path = answer.get_path()
      print(f'Number of moves: {len(path) - 1}')
      for state in path:
        if state.m:
          print(state.m, end=' - ')
      print('\n\n')

    return (answer, nb_tries,)

In [ ]:
def A_star(start: PuzzleState, heuristic):

  opened = PriorityQueue()
  closed = {}
  nb_tries = 0
  opened.insert(start, start.get_total_distance(heuristic))

  while not opened.empty():
    nb_tries += 1
    current = opened.min()

    if current.board == META:
      return (current, nb_tries,)

    current_hash = current.get_hash()
    if current_hash in closed and current.g >= closed[current_hash]:
      continue;

    closed[current_hash] = current.g
    for neighbor in current.get_neighbors():
      opened.insert(neighbor, neighbor.get_total_distance(heuristic))

In [ ]:
def best_first(start: PuzzleState, heuristic):

  opened = PriorityQueue()
  closed = {}
  nb_tries = 0
  opened.insert(start, start.get_total_distance(heuristic))

  while not opened.empty():
    nb_tries += 1
    current = opened.min()

    if current.board == META:
      return (current, nb_tries,)

    current_hash = current.get_hash()
    if current_hash in closed and current.g >= closed[current_hash]:
      continue;

    closed[current_hash] = current.g
    for neighbor in current.get_neighbors():
      opened.insert(neighbor, neighbor.get_heuristc(heuristic))

In [ ]:
board = [7,1,4,3,11,0,10,14,12,5,6,8,9,2,13,15]
state = PuzzleState(board)

In [ ]:
%time state.solve(A_star)

Number of tries: 592398
Number of moves: 46
b - b - r - r - t - t - t - l - l - b - r - b - l - l - t - r - r - b - r - b - l - t - l - l - t - t - r - r - b - r - b - l - l - t - l - t - r - r - b - l - b - l - b - r - r - r - 


CPU times: user 1min 3s, sys: 199 ms, total: 1min 4s
Wall time: 1min 4s


(<__main__.PuzzleState at 0x7912f566d690>, 592398)

In [ ]:
%time state.solve(best_first)

Number of tries: 384
Number of moves: 72
l - t - r - b - l - b - r - b - r - r - t - t - t - l - l - b - l - b - b - r - t - t - t - r - b - l - t - r - b - b - l - t - t - r - b - r - b - b - l - l - t - t - t - r - b - r - b - l - t - l - b - r - b - l - t - t - r - b - b - r - t - t - t - l - b - l - t - r - r - b - b - b - 


CPU times: user 56.1 ms, sys: 2.99 ms, total: 59.1 ms
Wall time: 58 ms


(<__main__.PuzzleState at 0x791302979810>, 384)